# Proxy Server — SafeAI as a REST API

SafeAI can run as a **sidecar service** alongside your application, exposing every guardrail
as a simple HTTP endpoint. Any language or framework — Go, Rust, Node, Java — can call it
over HTTP. No Python SDK required on the calling side.

This notebook uses FastAPI's `TestClient` to exercise the proxy in-process (no actual server
needed). In production you would run `safeai serve` and point your app at `http://localhost:8642`.

In [1]:
import json, tempfile
from pathlib import Path
from click.testing import CliRunner
from fastapi.testclient import TestClient
from safeai.cli.init import init_command
from safeai.proxy.server import create_app

tmp = tempfile.TemporaryDirectory()
work = Path(tmp.name)
CliRunner().invoke(init_command, ["--path", str(work)])

app = create_app(config_path=str(work / "safeai.yaml"), mode="sidecar")
client = TestClient(app)
print("Proxy app created in sidecar mode.")

Proxy app created in sidecar mode.


## 1. Health check

The health endpoint confirms the proxy is running and reports its mode and version.

In [2]:
resp = client.get("/v1/health")
print(f"Status code: {resp.status_code}")
print(json.dumps(resp.json(), indent=2))

Status code: 200
{
  "status": "ok",
  "mode": "sidecar",
  "version": "0.6.0"
}


## 2. Scan input via HTTP

POST text to `/v1/scan/input`. Clean text is allowed; text containing secrets is blocked.

In [3]:
# Clean text — should be allowed
resp_clean = client.post("/v1/scan/input", json={
    "text": "What is the capital of France?",
    "agent_id": "notebook-agent",
})
print("Clean input:")
print(json.dumps(resp_clean.json(), indent=2))

print()

# Text with a secret — should be blocked
resp_secret = client.post("/v1/scan/input", json={
    "text": "Use this key: sk-ABCDEF1234567890ABCDEF",
    "agent_id": "notebook-agent",
})
print("Secret input:")
print(json.dumps(resp_secret.json(), indent=2))

Clean input:
{
  "decision": {
    "action": "allow",
    "policy_name": "allow-input-by-default",
    "reason": "Allow when no restrictive policy matched."
  },
  "filtered": "What is the capital of France?",
  "detections": []
}

Secret input:
{
  "decision": {
    "action": "block",
    "policy_name": "block-secrets-everywhere",
    "reason": "Secrets must never cross any boundary."
  },
  "filtered": "",
  "detections": [
    {
      "detector": "openai_key",
      "tag": "secret.credential",
      "start": 14,
      "end": 39
    }
  ]
}


## 3. Scan structured payload

POST a nested JSON object to `/v1/scan/structured`. The scanner walks every leaf value
and reports detection paths so you know exactly where the problem is.

In [4]:
resp_struct = client.post("/v1/scan/structured", json={
    "payload": {
        "user": "alice",
        "config": {
            "api_key": "sk-ABCDEF1234567890ABCDEF",
            "region": "us-east-1",
        },
    },
    "agent_id": "notebook-agent",
})
print("Structured scan:")
print(json.dumps(resp_struct.json(), indent=2))

Structured scan:
{
  "decision": {
    "action": "block",
    "policy_name": "block-secrets-everywhere",
    "reason": "Secrets must never cross any boundary."
  },
  "filtered": null,
  "detections": [
    {
      "path": "$.config.api_key",
      "detector": "openai_key",
      "tag": "secret.credential",
      "start": 0,
      "end": 25
    }
  ]
}


## 4. Guard output

POST text to `/v1/guard/output`. PII (emails, phone numbers) is redacted and the
cleaned text is returned in `safe_output`.

In [5]:
resp_guard = client.post("/v1/guard/output", json={
    "text": "Contact alice@example.com or call 555-123-4567 for details.",
    "agent_id": "notebook-agent",
})
data = resp_guard.json()
print(f"Decision:    {data['decision']['action']}")
print(f"Safe output: {data['safe_output']}")
print(f"Detections:  {len(data['detections'])} items found")
for d in data["detections"]:
    print(f"  - {d['detector']}: {d['tag']}")

Decision:    redact
Safe output: Contact [REDACTED] or call [REDACTED] for details.
Detections:  2 items found
  - email: personal.pii
  - phone: personal.pii


## 5. Query audit log via API

Every scan and guard call is audited. Query the log for blocked events from our
previous calls.

In [6]:
resp_audit = client.post("/v1/audit/query", json={
    "action": "block",
    "limit": 10,
    "newest_first": True,
})
audit_data = resp_audit.json()
print(f"Blocked events: {audit_data['count']}")
for evt in audit_data["events"]:
    print(f"  [{evt.get('boundary', '?')}] {evt.get('policy_name', '?')} — {evt.get('reason', '')}")

Blocked events: 10
  [input] block-secrets-everywhere — Secrets must never cross any boundary.
  [input] block-secrets-everywhere — Secrets must never cross any boundary.
  [input] block-secrets-everywhere — Secrets must never cross any boundary.
  [action] secret-manager — capability token does not allow secret key 'OTHER_KEY'
  [action] block-confidential-action — Confidential data must not pass between agents.
  [action] block-secrets-everywhere — Secrets must never cross any boundary.
  [input] block-secrets-everywhere — Secrets must never cross any boundary.
  [input] block-secrets-everywhere — Secrets must never cross any boundary.
  [memory] memory-handle — memory handle agent binding mismatch
  [input] block-everything-on-input — Emergency lockdown — all input blocked.


## 6. Hot reload policies

Reload policies at runtime without restarting the proxy. Use `force: true` to reload
even if the files have not changed on disk.

In [7]:
resp_reload = client.post("/v1/policies/reload", json={"force": True})
print("Policy reload:")
print(json.dumps(resp_reload.json(), indent=2))

Policy reload:
{
  "reloaded": true,
  "mode": "force"
}


## 7. Memory via API

Write a value into SafeAI's secure memory store, then read it back.
Memory is scoped per agent and subject to the same policy enforcement.

In [8]:
# Write a value
resp_write = client.post("/v1/memory/write", json={
    "key": "session.context",
    "value": "User is asking about travel to Paris.",
    "agent_id": "notebook-agent",
})
print("Write:", json.dumps(resp_write.json()))

# Read it back
resp_read = client.post("/v1/memory/read", json={
    "key": "session.context",
    "agent_id": "notebook-agent",
})
print("Read: ", json.dumps(resp_read.json()))

Write: {"allowed": false}
Read:  {"value": null}


## 8. Metrics endpoint

The proxy exposes Prometheus-format metrics at `/v1/metrics` for monitoring
request counts, latencies, and policy decisions.

In [9]:
resp_metrics = client.get("/v1/metrics")
print(f"Content-Type: {resp_metrics.headers.get('content-type', 'unknown')}")
print()
# Show first 20 lines of Prometheus output
lines = resp_metrics.text.strip().splitlines()
for line in lines[:20]:
    print(line)
if len(lines) > 20:
    print(f"... ({len(lines) - 20} more lines)")

Content-Type: text/plain; charset=utf-8

# HELP safeai_proxy_requests_total Total proxy HTTP requests
# TYPE safeai_proxy_requests_total counter
safeai_proxy_requests_total{endpoint="/v1/audit/query",status="200",protocol="http"} 1
safeai_proxy_requests_total{endpoint="/v1/guard/output",status="200",protocol="http"} 1
safeai_proxy_requests_total{endpoint="/v1/memory/read",status="200",protocol="http"} 1
safeai_proxy_requests_total{endpoint="/v1/memory/write",status="200",protocol="http"} 1
safeai_proxy_requests_total{endpoint="/v1/policies/reload",status="200",protocol="http"} 1
safeai_proxy_requests_total{endpoint="/v1/scan/input",status="200",protocol="http"} 2
safeai_proxy_requests_total{endpoint="/v1/scan/structured",status="200",protocol="http"} 1
# HELP safeai_proxy_decisions_total Total proxy decisions by action
# TYPE safeai_proxy_decisions_total counter
safeai_proxy_decisions_total{endpoint="/v1/audit/query",action="allow"} 1
safeai_proxy_decisions_total{endpoint="/v1/guard/ou

In [10]:
# Cleanup
tmp.cleanup()
print("Temporary directory cleaned up.")

Temporary directory cleaned up.


---

## Summary

The SafeAI proxy exposes every guardrail as a REST endpoint. Two deployment modes:

- **Sidecar mode** — runs next to your application. Your app calls the proxy to scan
  inputs, guard outputs, and manage memory. You control when and what gets checked.
- **Gateway mode** — sits between your app and the upstream LLM. Every request is
  automatically scanned on the way out, and every response is guarded on the way back.
  Requires `source_agent_id` and `destination_agent_id` on multi-agent endpoints.

To run the proxy in production:

```bash
# Sidecar (default)
safeai serve --mode sidecar --port 8642

# Gateway with upstream
safeai serve --mode gateway --upstream-url https://api.openai.com
```

Any HTTP client in any language can then call `http://localhost:8642/v1/scan/input`,
`/v1/guard/output`, etc.